# Week 6: File I/O, Filtering, and Pandas

## Learning to use the ‘pandas‘ Python library

In this notebook we use the `pandas` python library to perform file input and output and to filter and organize our data. We have used `pandas` in every lecture so far without going into much detail because the functions it provides are too helpful to wait until Week 6. `pandas` is one of the most widely used python packages because it makes it easy to perform a large number of standard operations with tables of data. The library is extremely well documented with tutorials: https://pandas.pydata.org/docs/getting_started/index.html

From this point forward, we will be working with a single dataset which we will progressively transform, analyse and visualise from Week 6 to Week 9. That dataset is the combined comment sections of two Reddit posts collected using the `praw` API that we saw in Week 2. The first post is about cloning [wooly mammoths](https://www.reddit.com/r/technology/comments/10py29t/scientists_are_reincarnating_the_woolly_mammoth/), and the second is about [remote work](https://www.reddit.com/r/technology/comments/10plq1f/remote_work_hasnt_actually_saved_americans_much/). The notebook I used to download the data into a file is available on Canvas, though you are welcome to just use the JSON file provided.

This week, we will do some basic filtering and slicing to get an idea of what the dataset is like. We will also learn about using regular expressions, often called `regex`, to perform more general searches of text. Finally, we will save the data into a new file which we will start from next week.


In [1]:
%%capture
# Run this cell to install the pandas library using the Python package manager pip
# You probably already have pandas as it comes with Anaconda by default
!pip install pandas

In [2]:
import json
import pandas as pd
import re
from datetime import datetime

## Reading and Writing Files

As you've seen in the other classes already, web data can come in a variety of formats. We've already dealt with JSON, CSV, and HTML files, which are the ones you will run into most frequently with internet data. These file types tell you something about the way the information within them is stored, which allows a computer to know how to read that information back into a form Python can use.

### JSON

JSON stands for JavaScript Object Notation. It is a simple file type in which the data is stored as a string which is readable in any text editor. JSON files look very similar to Python dictionaries, and follow four basic rules. Every time there is an object with some set of attributes, it is enclosed in curly brackets `{}`. The attributes of the object are stored as key:value pairs separated by a colon, with commas between attributes. When an attribute takes a list as the value, the list is enclosed in square brackets `[]`. These rules can be combined and repeated to form very large objects, but a small one is shown below. Reading and writing JSON files can be done with the base Python libraries.


In [3]:
json_example = {"key": "value", "key2": ["value", "value", "value"]}

<div class="alert alert-info">

**Exercise 0.1:** Write a JSON file from the dictionary above, and then read it back again. Call the file `test.json`.
</div>

The standard way to read and write files in python is to start by getting a file. Use `with open(filename, mode) as variable:`
to create a variable where you can access the file. `with` is a special helper function for opening files that makes sure they close when you're done and don't get corrupted (or at least tries to). The value you give for `mode`, in this case `w` controls what you can do with the file. The table below shows the options for the mode.


| mode (name)             | Function | 
|:------------------------|:------------|
| `w` (write)             | Open (or create and open) the file, erase any contents, and write new data |                    
| `a` (append)            | Open (or create and open) the file, keep any contents, and write new data  |
| `r` (read)              | Open the file and read the contents. Fails if the file does not exist|
| `x` (create)            | Create a new file. Fails if the file already exists|


In [4]:
# first we create and write to the file
with open("test.json", "w") as f:
    # json.dump takes a variable and writes it to the file given in the second argument
    json.dump(json_example, f)

In [5]:
# now we read it back using very similar code
with open("test.json", "r") as f:
    json_loaded = json.load(f)

In [6]:
# To check if it worked, see if our loaded data matches what we saved!
json_loaded == json_example

True

### CSV

CSV stands for Comma Separated Values. It is a simple file type in which the data is stored as a string which is readable in any text editor. CSV files are the basic version of Excel files, and are used for tabular data. As you might guess, the values are separated by commas. The only other rule is that new rows are written on new lines of the file. Sometimes, the first row has headers and the first column has row labels. If so, you can tell `pandas` to use these as row and/or column labels.


<div class="alert alert-info">

**Exercise 0.2:** Write a CSV file from the dataframe below, and then read it back again. Call the file `test.csv`.
</div>

In [7]:
csv_example = pd.DataFrame(['value','value','value'])
csv_example

,0
0,value
1,value
2,value


In [8]:
# pandas does all the work for us
# the only trick here is that since we don't care about the dataframe index, we don't save it

with open('test.csv', 'w') as f:
    csv_example.to_csv(f, index=False)

In [9]:
# pandas does all the work for us

with open('test.csv', 'r') as f:
    csv_loaded = pd.read_csv(f)

In [10]:
# checking that we got the same thing as before
csv_loaded

,0
0,value
1,value
2,value


## TXT

Text files are extremely simple files. They contain text written out in plain form. You can store JSON or CSV files as TXT files, but there isn't usually a good reason to since the other file types provide more detail on how to read them back. The nice thing about txt files is that you can write anything that can be stored as as string. Therefore, you can build your own file types on top of txt files.

<div class="alert alert-info">

**Exercise 0.3:** Write a txt file from the string below, and then read it back again. Call the file `test.txt`.
</div>

In [11]:
txt_example = "value"

In [12]:
# if you use a with block you can just use f.write to add a string
with open('text.txt','w') as f:
    f.write(txt_example)

In [13]:
# f.read returns everything in the file as a string
# if that is too much data there are methods for going in shorter pieces
with open('text.txt','r') as f:
    txt_loaded = f.read()

In [14]:
# check that it worked
txt_example == txt_loaded

True

### Exercise 1: Loading our Reddit data from JSON

<div class="alert alert-info">

**Exercise 1.1:** Use the methods we have just demonstrated to read our JSON data from the file. Store it as a dictionary for now.
</div>

In [15]:
with open('reddit_data.json','r') as f:
    reddit_data = json.load(f)

In [16]:
# if you succeeded, you should have a dicitonary with two keys:
# ['10py29t', '10plq1f']
# these are the IDs of the Reddit posts we are using

reddit_data.keys()

dict_keys(['10py29t', '10plq1f'])

<div class="alert alert-info">
<b>Exercise 1.1a:</b> What data do we have about each post?
</div>

In [17]:
reddit_data['10py29t'].keys()

dict_keys(['title', 'created_at', 'id', 'permalink', 'num_comments', 'score', 'upvote_ratio', 'external_link', 'comments'])

<div class="alert alert-info">
<b>Exercise 1.1b:</b> What data do we have about each comment?
</div>

In [18]:
reddit_data['10py29t']['comments'][0].keys()

dict_keys(['body', 'created_at', 'id', 'parent', 'score'])

<div class="alert alert-info">
<b>Exercise 1.1c:</b> Now we can see how many comments we have:
</div>

In [19]:
# Using what you know about navigating dictionaries, find the comments under each key and count them
for key in reddit_data.keys():
    print(f'Post {key} has {len(reddit_data[key]["comments"])} comments.')

Post 10py29t has 915 comments.
Post 10plq1f has 817 comments.


## Pandas

Now that we have the data into a dictionary, let's start to talk about `pandas`. Pandas comes from the words PANel DAta, a term from economics used for time-series data. It was originally started at a hedge fund, so at least one good thing has come out of quantitative finance. Because of the focus on time series, pandas has two main types of data, `Series` and `DataFrame`. A `Series` is a single column of data with an index, originally a series of observations of a single variable at different points in time. A `DataFrame` is a collection of `Series` which share an index, making what resembles a table.

In [20]:
# here is a simple series
series_example = pd.Series([1,2,3], index = ['x','y','z'])
series_example

x    1
y    2
z    3
dtype: int64

In [21]:
# and a simple dataframe
df_example = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], columns=['a','b','c'],  index = ['x','y','z'])
df_example

,a,b,c
x,1,2,3
y,4,5,6
z,7,8,9


Each series (column) in a dataframe has a name which can be read across the top of the dataframe (a,b,c in the example above). The left hand column is called the `index` and is shared across all the series in the dataframe. Entries in a series or dataframe can be accessed in several ways:

<div class="alert-info alert">

**Example 2.0:** Several ways to access a series or dataframe are shown below:

</div>

In [22]:
# by the index in a series
series_example[0]

1

In [23]:
# by the index values using .loc
series_example.loc['x']

1

In [24]:
# by the position in the series using .iloc
series_example.iloc[0]

1

In [25]:
# with booleans indicating which values to include
# this requires one boolean per row

series_example[[True, False, True]]

x    1
z    3
dtype: int64

In [26]:
# all of these also accept lists of entries or slices as with python lists

print(series_example[0:2])

print(series_example.loc[['x','y']])

print(series_example.iloc[1:3])

x    1
y    2
dtype: int64
x    1
y    2
dtype: int64
y    2
z    3
dtype: int64


In [27]:
# Similarly, DataFrames can be accessed using most of these methods, but require both rows and columns:
# rows come before columns
print(df_example.loc['x','a'])

# you can only use positional indices with iloc
# this won't work df_example[0,0]
print(df_example.iloc[0,0])

1
1


In [28]:
# and you can mix and match slices and indices

print(df_example.loc[['x','y'],'a'])

print(df_example.iloc[0:3,1])

x    1
y    4
Name: a, dtype: int64
x    2
y    5
z    8
Name: b, dtype: int64


<div class="alert-info alert">

**Example 2.1:** Now try it yourself. Find the largest element in the `example_series` and `example_df` using each of the methods shown above. Then for the dataframe, find the row and column with the largest sum.
</div>

In [29]:
# for the series
print(series_example[2])
print(series_example.loc['z'])
print(series_example.iloc[2])

3
3
3


In [30]:
# for the df
print(df_example.loc['z','c'])
print(df_example.iloc[2,2])

9
9


In [31]:
# for the row and column
print(df_example.loc[:,'c'])
print(df_example.loc['z',:])

x    3
y    6
z    9
Name: c, dtype: int64
a    7
b    8
c    9
Name: z, dtype: int64


### Booleans

The most powerful way to use the filtering and slicing is with logical statements. Since we can extract elements of a series or dataframe based on booleans, if we can create boolean series of what we want, it is easy to access. 

<div class="alert-info alert">

**Example 3.0:** If we want to find the even numbers in a series, we can first make a filter for which numbers are even, and then apply it to the series:
</div>

In [32]:
evens_filter = series_example %2 == 0 
evens_filter

x    False
y     True
z    False
dtype: bool

In [33]:
series_example[evens_filter]

y    2
dtype: int64

In [34]:
# typically you will see this done in a single step:
series_example[series_example%2 == 0]

y    2
dtype: int64

In [35]:
# you might also want to know the values in the other columns of a dataframe when a condition is true in the first column

df_example[df_example['a']%2==0]

,a,b,c
y,4,5,6


<div class="alert-info alert">

**Example 3.1:** Find the rows of the `df_example` where the entry in column `b` is even.
</div>

In [36]:
df_example[df_example['b']%2==0]

,a,b,c
x,1,2,3
z,7,8,9


## Exercise 2: Loading our Reddit data into Pandas and filtering it

Using the skills we've just covered, we want to load our dictionary of Reddit data into a pandas dataframe with one row per comment and columns for each propety of the comments. Also include a column for the post to which the comment is responding.

<div class="alert-info alert">

**Excercise 2.1:** Convert the `reddit_data` dictionary into a dataframe
</div>

In [37]:
# you'll want to start with an empty list
reddit_comments = []

# loop over the two posts
for key in reddit_data.keys():
    # loop over the comments in the posts
    for comment in reddit_data[key]['comments']:
        # add an entry for which post it came from
        comment['post'] = key
        # add the comment to the list of all comments
        reddit_comments.append(comment)

# convert to a dataframe and cleanup the datetime
reddit_df = pd.DataFrame(reddit_comments)
reddit_df['created_at'] = pd.to_datetime(reddit_df['created_at'], unit='s')

In [38]:
# We can see how many rows and columns there are. There should be one row per comment, 915 + 817 = 1732
reddit_df.shape

(1732, 6)

In [39]:
# we also seem to have all of the columns we wanted
reddit_df.head()

,body,created_at,id,parent,score,post
0,when you say it like that - gimme 2,2023-02-01 07:42:00,j6r1s1s,t1_j6qxv18,6,10py29t
1,Mammork?\n\nI would that make them hammoths?,2023-02-01 10:18:59,j6rd2o8,t1_j6qxv18,3,10py29t
2,Mmmmmmmm mammoth pork,2023-02-01 06:52:37,j6qxv18,t1_j6obhxv,22,10py29t
3,It’ll just be made from pressed pork.,2023-01-31 19:16:08,j6obhxv,t1_j6notr0,68,10py29t
4,11 years before they’re extinct again,2023-02-01 06:51:58,j6qxt2e,t1_j6notr0,7,10py29t


<div class="alert-info alert">

**Excercise 2.2:** Now that we've made the dataframe, let's filter it based on:
- which post it came from
-when the comment was made
    
</div>

In [40]:
# First get all the mammoth posts
reddit_df[reddit_df['post']=='10py29t']

,body,created_at,id,parent,score,post
0,when you say it like that - gimme 2,2023-02-01 07:42:00,j6r1s1s,t1_j6qxv18,6,10py29t
1,Mammork?\n\nI would that make them hammoths?,2023-02-01 10:18:59,j6rd2o8,t1_j6qxv18,3,10py29t
2,Mmmmmmmm mammoth pork,2023-02-01 06:52:37,j6qxv18,t1_j6obhxv,22,10py29t
3,It’ll just be made from pressed pork.,2023-01-31 19:16:08,j6obhxv,t1_j6notr0,68,10py29t
4,11 years before they’re extinct again,2023-02-01 06:51:58,j6qxt2e,t1_j6notr0,7,10py29t
...,...,...,...,...,...,...
910,“The World Poaching Association in a press rel...,2023-02-01 13:05:13,j6rrmwl,t3_10py29t,1,10py29t
911,Our hunting practices probably made them go ex...,2023-02-01 13:26:41,j6ru3om,t3_10py29t,1,10py29t
912,Why?,2023-02-01 13:35:55,j6rv81a,t3_10py29t,1,10py29t
913,Did Jurassic Park teach us nothing? 😂,2023-02-01 14:23:26,j6s1ib8,t3_10py29t,1,10py29t


In [41]:
# Now let's get the others by filtering for `not mammoth posts`
reddit_df[~(reddit_df['post']=='10py29t')]

,body,created_at,id,parent,score,post
915,"I'm there where you were, in the elements and ...",2023-01-31 16:54:59,j6nobr5,t1_j6nneu9,2,10plq1f
916,I was able to leverage my experience and the l...,2023-01-31 23:12:07,j6pd2pm,t1_j6p4jgg,2,10plq1f
917,As someone who also does telecom installation ...,2023-01-31 22:15:51,j6p4jgg,t1_j6nneu9,2,10plq1f
918,Hey man I get it. I spent nearly a decade inst...,2023-01-31 16:49:25,j6nneu9,t1_j6nerw8,6,10plq1f
919,"When you apply to WFH jobs, be reasonable, but...",2023-01-31 22:08:51,j6p3go5,t1_j6nerw8,2,10plq1f
...,...,...,...,...,...,...
1727,Show this to your employer when they try to br...,2023-02-01 14:24:01,j6s1le2,t3_10plq1f,1,10plq1f
1728,Are projects more successful working remotely ...,2023-02-01 14:51:21,j6s5iww,t3_10plq1f,1,10plq1f
1729,Business Insider is literal corporate trash.,2023-02-01 14:53:10,j6s5sx8,t3_10plq1f,1,10plq1f
1730,"More money, less commute.",2023-02-01 15:15:14,j6s94yv,t3_10plq1f,1,10plq1f


In [42]:
# how many posts were made in the first hour of each post?

# the mammoth post was made at Tuesday, January 31, 2023 12:42:36 PM
# the remote work post was made at Tuesday, January 31, 2023 2:32:31 AM
 
mammoth_date = datetime.strptime('31/01/23 13:42:36', '%d/%m/%y %H:%M:%S')
remote_work_date = datetime.strptime('31/01/23 15:32:31', '%d/%m/%y %H:%M:%S')

print(f"The mammoth post had {len(reddit_df[(reddit_df['post'] == '10py29t') & (reddit_df['created_at'] < mammoth_date)])} comments in the first hour.")

print(f"The remote work post had {len(reddit_df[~(reddit_df['post'] == '10py29t') & (reddit_df['created_at'] < remote_work_date)])} comments in the first hour.")

The mammoth post had 4 comments in the first hour.
The remote work post had 573 comments in the first hour.


<div class="alert-info alert">

**Excercise 2.3:** We're planning to do sentiment analysis on the text in these comments. Do you notice anything in the comments below?
    
</div>

In [43]:
# these might pose problems for us
reddit_df['body'][[19,63,119,929,1157,1457]]

19                                           You think? 🧐
63      Offer it in the Natural History Museum's cafet...
119     Breaking News 🚨 🚨 🚨\n\nWoolly Mammoth to be br...
929     >I want this for everyone! \n\nToo bad only th...
1157                          Take my poor woman's gold 🥇
1457    Business Insider definitely wants you back in ...
Name: body, dtype: object

<div class="alert-info alert">

**Excercise 2.4:** For now, let's find all the rows with special characters in them and create a new column with a boolean label called `has_emoji`.
    
</div>

### Regular Expressions

To find emojis, we're going to use a special type of searching in strings called `regular expressions`. "Regex", as it is commonly called, is a standard language for expressing text searches which can be used across programming languages (and elsewhere like in Excel). Beyond what I will explain, an excellent in-depth tutorial is available here (https://regexone.com/).

Regex uses pattern matching between a search string and a set of target strings. It can return either whether or not there was a match or what the match was if it existed. The most basic version is just like using `find` on your computer. If the thing you search is anywhere in the target string, it will match:

In [44]:
# searches for 'abc' in the target strings
# the syntax is re.search('search expression', 'target expression')

targets = ['abc123', '123abc', 'acb132']

search = 'abc'

[True if re.search(search, target) else False for target in targets] 

[True, True, False]

The next level of regex is `wildcards`. The `.` character matches anything. You can also have a list of options in square braces `[]`:

In [45]:
# searches for a_c where _ can be any one character

targets = ['aac', 'abc', 'abbc']

search = 'a.c'

[True if re.search(search, target) else False for target in targets] 

[True, True, False]

In [46]:
# searches for a_c where _ can be any one character from the set [abc]

targets = ['aac', 'adc', 'abbc']

search = 'a[abc]c'

[True if re.search(search, target) else False for target in targets] 

[True, False, False]

You might want to allow for more than one copy of a character or wildcard, or none at all. The `*` character matches zero or more repetitions, while the `+` character matches one or more.

In [47]:
# searches for a_c where _ can be any one or more character from the set [abc]

targets = ['aac', 'adc', 'abbc']

search = 'a[abc]+c'

[True if re.search(search, target) else False for target in targets] 

[True, False, True]

In [48]:
# searches for a_c where _ can be anything at all

targets = ['aac', 'a$c', 'bbca']

search = 'a.*c'

[True if re.search(search, target) else False for target in targets] 

[True, True, False]

You can exclude characters using the `^` and `[]` in combination.

In [49]:
# searches for a_c where _ can be anything except `b`

targets = ['aaac', 'a$c', 'abc', 'ac']

search = 'a[^b]*c'

[True if re.search(search, target) else False for target in targets] 

[True, True, False, True]

Two last things: sometimes you might want to match a character that already has a meaning (like `*`), then you use a `\` to escape the character in question. Also, if you want to allow a range of characters, you can use a `-`. (This only works for alphanumerics). All of these features can be combined to match very complex things. https://www.explainxkcd.com/wiki/index.php/1313:_Regex_Golf

In [50]:
# searches for a_c where _ can only be `*`

targets = ['aaac', 'a$c', 'abc', 'ac']

search = 'a\$c'

[True if re.search(search, target) else False for target in targets] 

[False, True, False, False]

In [51]:
# searches for a_c where _ can be any set of at least one lowercase letter(s) 

targets = ['aaac', 'a$c', 'abc', 'ac']

search = 'a[a-z]+c'

[True if re.search(search, target) else False for target in targets] 

[True, False, True, False]

<div class="alert-info alert">

**Excercise 2.4a:** Create a regex search which matches `c_b_a` where `_` can be any one uppercase letter. 
    
</div>

In [52]:
targets = ['cba', 'cBBbAa', 'cCbBa', 'ccbba']

search = 'c[A-Z]b[A-Z]a'

[True if re.search(search, target) else False for target in targets] 

[False, False, True, False]

<div class="alert-info alert">

**Excercise 2.4b:** Find a string which matches the regex below. (Hint: the name I used might help.)
    
</div>

In [53]:
target = "🧐"

regex_nonstandard_chars = '''[^a-zA-Z0-9\s\-‘’“”'"…:;?!.·,|/\\\(\)\[\]=≈\*^+%&@°#$><—–~_]+'''

re.search(regex_nonstandard_chars, target)

<re.Match object; span=(0, 1), match='🧐'>

<div class="alert-info alert">

**Excercise 2.4c:** Using regex, find all the entries in `reddit_df` where the text includes an emoji.
    
</div>

In [54]:
# pandas also has special behaviours for regex. rather than using re.search, we 
# use Series.str.contains('search string')

reddit_df['body'].str.contains('a')

0        True
1        True
2        True
3        True
4        True
        ...  
1727     True
1728    False
1729     True
1730    False
1731     True
Name: body, Length: 1732, dtype: bool

In [55]:
# This uses my nonstandard characters regex to find the emoji

reddit_df[reddit_df['body'].str.contains(regex_nonstandard_chars)]['body']

19                                           You think? 🧐
63      Offer it in the Natural History Museum's cafet...
119     Breaking News 🚨 🚨 🚨\n\nWoolly Mammoth to be br...
138     We’ve had second extinction, but what about th...
149     I'm crying I'm laughing so hard!!!! ...Now jus...
162              Mfw a mammoth scolds me about politics 🦣
167     That's interesting, I wonder if they were able...
307                      Ayo we *breeding* raptors? 😳 /jk
343                      We need more drowning mammoths 🦣
473     Did we not learn anything from Jurassic Park?! 😳😬
480       Lol, they can go roam freely in Antartica. 🦣🦣❄❄
585     Bringing it back to a hot and getting hotter p...
648     So we’re going to have mammoth steaks while th...
683     Why do we fuck with things that shouldn’t be f...
697     The forbidden hamburger meat. They couldn't se...
717     Human stupidity at his maximum level. why not ...
833     *sigh*…just because they could doesn’t mean th...
843     Time t

<div class="alert-info alert">

**Excercise 2.5:** Add a column to `reddit_df` which indicates whether the body text of the comment contains an emoji.
    
</div>

In [56]:
reddit_df['has_emoji'] = reddit_df['body'].str.contains(regex_nonstandard_chars)

In [1]:
sample_text = '''Husband (38M) and I (39F) have been trying for a baby for around 2 years now. We've had various tests and my results have been fine but there are a couple of issues on his side. He's being very good and taking supplements etc but he keeps trying to push me down the ivf route.

We've been to the clinic for tests and now the next step would be starting the procedure. I have said I am not comfortable with it. I have anxiety and quite frankly the thought of the whole process sends me into a panic attack. I am waking up with a racing heart every morning. I feel sick and unhappy. I feel like I'm walking on eggshells to avoid the conversation.

I don't want to regret not having a child but at the same time I wish the whole thing would just go away.

I don't know what to do. He says his mental health will be affected if we don't have a kid but mine is already affected now.

Any advice?'''

In [2]:
import re

In [ ]:
'\([0-9][0-9][MFmf]'

In [9]:
search = '[I] [\(\[][0-9][0-9][MFmf][\)\]]'

re.search(search, sample_text)

<re.Match object; span=(18, 25), match='I (39F)'>

### Exercise 3

<div class="alert-info alert">

Split the `reddit_df` into two dataframes and save each one as a CSV file called `mammoths.csv` and `remote_work.csv`

</div>



In [57]:
# Split the dataframe
mammoths_df = reddit_df[reddit_df['post']=='10py29t']
remote_work_df = reddit_df[reddit_df['post']=='10plq1f']

# save each dataframe
mammoths_df.to_csv('mammoths.csv',index=False)
remote_work_df.to_csv('remote_work.csv',index=False)